In [48]:
import wandb
import torch
import torch.nn 
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import numpy as np 
from PIL import Image
from models.models import *
import gensim.downloader as api
from tqdm.notebook import tqdm
from utils.utils import *

In [49]:
data_path = r"C:\Users\Joan\Desktop\Deep_Learning_project\features\data" # Path of the data folder from the dataset
anotation_path= r"C:\Users\Joan\Desktop\Deep_Learning_project\dlnn-project_ia-group_15\anotations_keras.pkl" # Path of the file with the words spotted in the images
img_dir = data_path + r"\JPEGImages"
txt_dir = data_path + r"\ImageSets\0"

In [50]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # Set device to GPU if available

In [52]:
# Transformations of the data
input_size = 224    
data_transforms_train = torchvision.transforms.Compose([
    torchvision.transforms.Resize(236, interpolation=torchvision.transforms.InterpolationMode.BICUBIC),
    torchvision.transforms.RandomResizedCrop(input_size),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data_transforms_test = torchvision.transforms.Compose([
        torchvision.transforms.Resize(236, interpolation=torchvision.transforms.InterpolationMode.BICUBIC),
        torchvision.transforms.CenterCrop(input_size),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [53]:
# w2v = api.load('glove-wiki-gigaword-300') # Initialize the embeding

# ocr_data = pd.read_pickle(anotation_path) # Open the data with the data of the OCR

In [54]:
train_img_names, y_train, test_img_names, y_test, val_img_names, y_val = load_labels_and_split(txt_dir) # Load labels and split them into train, test and validation sets, with a 60% train, 20% validation 20% test split 
# Creating the datasets and the loaders for the train, test and validation
# Train
train_dataset = Dataset_imgs(img_dir, train_img_names, y_train, transform=data_transforms_train)
train_loader = make_loader(train_dataset, 1, shuffle=False) # Its not necessary to shuffle this train set as we are not training any model, we are only extracting features
# Test
test_dataset = Dataset_imgs(img_dir, test_img_names, y_test, transform=data_transforms_test)
test_loader = make_loader(test_dataset, 1)
# Validation
val_dataset = Dataset_imgs(img_dir, val_img_names, y_val, transform=data_transforms_test)
val_loader = make_loader(val_dataset, 1)

# Make the model
model = Feature_Extractor().to(device)

In [55]:
def extract_features(loader, model):
    # Passes the images through the model and returns the features, labels and image names
    model.eval()
    features = []
    labels = []
    img_names = []
    for label, img, img_name in tqdm(loader):
        img = img.to(device)
        label = label.to(device)
        feature = model(img)
        features.append(feature.squeeze().cpu().numpy())
        labels.append(int(label.cpu().numpy()))
        img_names.append(img_name[0])
    return features, labels, img_names

In [56]:
print("Extracting features from the validation data:")
val_features, val_labels, val_img_names = extract_features(val_loader, model)

Extracting features from the validation data:


  0%|          | 0/4851 [00:00<?, ?it/s]

In [57]:
print("Extracting features from the train data:")
train_features, train_labels, train_img_names = extract_features(train_loader, model)

Extracting features from the train data:


  0%|          | 0/14553 [00:00<?, ?it/s]

In [58]:
print("Extracting features from the test data:")
test_features, test_labels, test_img_names = extract_features(test_loader, model)

Extracting features from the test data:


  0%|          | 0/4851 [00:00<?, ?it/s]

In [ ]:
train_data = {"features": train_features, "labels": train_labels, "img_names": train_img_names}
val_data = {"features": val_features, "labels": val_labels, "img_names": val_img_names}
test_data = {"features": test_features, "labels": test_labels, "img_names": test_img_names}
data = {"train": train_data, "val": val_data, "test": test_data}

In [ ]:
import pickle
# Save the features extracted in a pickle file
with open("features_extracted.pkl", "wb") as f:
    pickle.dump(data, f)